In [1]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [111]:
study='patient7'

In [112]:
rp=dcm.read_file('/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/dicom/rp_'+study+'.dcm')
rd=dcm.read_file('/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/dicom/rd_'+study+'.dcm')

# Create an mhd file from DICOM RT dose 

In [113]:
#get pixel array
rdpix=rd.pixel_array.astype(np.float32)

#Convert rt dose value into Gy value 
f=float(rd.DoseGridScaling)
rdpix1=rdpix*f 

#create an sitk image from this array
rdim=sitk.GetImageFromArray(rdpix1)

#Set origin of image from rt dose value
origin=[]
for val in rd.ImagePositionPatient: origin.append(float(val))
rdim.SetOrigin(origin)
#Set spacing of image from rt dose value (x et y)
spacing=[]
for val in rd.PixelSpacing: spacing.append(float(val))
#for z direction
spacez=float(rd.GridFrameOffsetVector[1]-rd.GridFrameOffsetVector[0])
spacing.append(spacez)
rdim.SetSpacing(spacing)

#write rd image in '.raw'
sitk.WriteImage(rdim, '/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/results/emc_'+study+'.mhd')
print('emc_'+study+'.mhd --> ok')

emc_patient7.mhd --> ok


# GATE output and RT dose DICOM

In [114]:
#Get the rd raw created  and gate image
refim=sitk.ReadImage('/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/images/emc_'+study+'.mhd')
gateim=sitk.ReadImage('/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/results/out_'+study+'.mhd')
# get pixels arrays of images and vizualization
gatepix=sitk.GetArrayFromImage(gateim)
refpix=sitk.GetArrayFromImage(refim)                                          
ct_viewer(refpix)
ct_viewer(gatepix)

interactive(children=(IntSlider(value=69, description=u'myslice', max=138), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=150, description=u'myslice', max=300), Output()), _dom_classes=('widget-interact',))

In [115]:
# compute true value gate output offset in the DICOM frame
ct_im=sitk.ReadImage('/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/images/'+study+'.mhd')

half_ct_im=[]
half_gateim=[]
void_actor=[]
new_origin=[]

for size, space in zip(ct_im.GetSize(), ct_im.GetSpacing()): half_ct_im.append((size*space)/2)
for size, space in zip(gateim.GetSize(), gateim.GetSpacing()): half_gateim.append((size*space)/2)

for vala, valb  in zip(half_gateim, half_ct_im): void_actor.append(vala-valb)
    
ct_im_origin=[ct_im.GetOrigin()[0], ct_im.GetOrigin()[1]*-1, ct_im.GetOrigin()[2]*-1]

for val1, val2 in zip(ct_im_origin, void_actor): new_origin.append(val1-val2)
    
gateim.SetOrigin(new_origin)
print('New Origin : ', gateim.GetOrigin())

New Origin :  (-307.61725, -371.0938, -200.0)


In [116]:
#to mask GATE images an get just BODY DOSE
#Resample (Downsampling) rd image into gate image size an resolution
refim_res=sitk.Resample(refim, gateim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
refim_respix=sitk.GetArrayFromImage(refim_res)
ct_viewer(refim_respix)

interactive(children=(IntSlider(value=150, description=u'myslice', max=300), Output()), _dom_classes=('widget-interact',))

In [117]:
#Create a boolean mask to apply on Gate image (remove dose outside the patient according to RTdose)
mask=np.ma.make_mask(refim_respix)
#apply on gate pixel array
gatepix_mask=gatepix*mask
ct_viewer(gatepix_mask)

interactive(children=(IntSlider(value=150, description=u'myslice', max=300), Output()), _dom_classes=('widget-interact',))

# Absolute dose conversion 

In [118]:
energy=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy
if energy == 6:
    GVref=0.000108726
    Npart=1033128667.0
    DrG=0.000108726
    
if energy == 9:
    GVref=0.000140945
    Npart=1136273663.0
    DrG=0.000140945

if energy == 12:
    GVref=0.000150559
    Npart=1086793583.0
    DrG=0.000150559
print(energy)

6


In [119]:
#Absolute dose conversion LUC SIMON
um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

gatepix_abs=gatepix_mask*(1.0/GVref)*0.01*um*nFrac
print(gatepix_abs.max())

2.11885


In [120]:
#Absolute dose conversion BIBLIO
GperPart=DrG/Npart

Dref=0.01
F=Dref/GperPart

um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

gatepix_abs=(gatepix_mask/Npart)*F*um*nFrac
print(gatepix_abs.max())

2.11885


In [121]:
#Create a raw image of gate with mask and a raw image from resampled rt dose dicom 
gate_dose=sitk.GetImageFromArray(gatepix_abs)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
sitk.WriteImage(refim_res, '/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/results/emc_resampled_'+study+'.mhd')
sitk.WriteImage(gate_dose, '/media/sf_CRCT/JupyterNotebook/data/image/curie/clinic/'+study+'/results/gate_'+study+'.mhd')